## EXAM SCHEDULER PROTOTYPE

In [22]:
import pandas as pd
from collections import defaultdict
import os

### CONSTANTS

In [23]:
SEM = '2024-2025-1'
# SEM = '2023-2024-2'
DATA_PATH = './data/'

### Loading the data

First we will create a map from course code to course name. We are going to use course name as the key.

Below code will search for all `'{SEM}_{dept}.csv'` files in `DATA_PATH` directory. It will ignore `{SEM}_report.csv` as it is assumed to have the data for relation between all the students and the course names.

In [24]:
courses : dict[str, str] = {}

for file in os.listdir(DATA_PATH):

    if file.startswith(SEM):

        if file == f'{SEM}_report.csv':
            continue

        df = pd.read_csv(DATA_PATH + file, usecols= [0, 1], index_col=0)
        courses.update(
            df['cname'].to_dict()
        )

In [25]:
enrolled : dict[str, set] = defaultdict(set)

with open(DATA_PATH + SEM + '_report.csv') as f:

    f.readline()

    for line in f:

        line = line.strip().split(',')
        c_num = line[4].lstrip('"').rstrip('"')
        c_name = courses.get(c_num, None)
        if c_name is None:
            c_name = line[5].lstrip('"').rstrip('"')
            courses[c_num] = c_name

        enrolled[c_name].add(int(line[0]))

Printing all the courses that I am enrolled in

In [26]:
for course in enrolled:
    if 22016 in enrolled[course]:
        print(course)

Law Relating to Intellectual Property and Patents
Fundamentals of Database Systems
Computer Vision
Machine Learning
Real Analysis I
Theory of Computation
EECS Laboratory I


### Creating the graph

We use the networkx library for graph processing.

In [27]:
import networkx as nx

In [28]:
g = nx.Graph()

We use the course names as the vertice and draw an edge between them if there is a common student between any two course.

In [29]:
g = nx.Graph()

for course in enrolled:
    for course2 in enrolled:

        if course == course2:
            continue

        if course in g and course2 in g[course]:
            continue

        if not enrolled[course].isdisjoint(enrolled[course2]):
            g.add_edge(course, course2)

Now, getting number of slots for exams becomes same question as number of colors our graph can be colored in :)

In [30]:
colors = nx.coloring.greedy_color(g)

In [31]:
max_colours = max(colors.values())

print("Graph colored in total of", max_colours + 1, "colors")

Graph colored in total of 18 colors
